<a href="https://colab.research.google.com/github/ricardomsanti/douExplore/blob/master/c_pia_de_dex_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DEPENCENCY 

In [1]:
!pip install html5lib
!pip install pyspark
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 27.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=9f22b69322ccd8b8e8e2edc3964f6f3fece735d79863c4f0070affb536806e88
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 18.5 MB/s eta 0:00:00


In [ ]:

url = "https://www.jusbrasil.com.br/diarios/DOU/2023/01/18/Secao-1"
response = r.get(url)
from bs4 import BeautifulSoup as soup
soup = BeautifulSoup(response.text, "html5lib")
for x in soup.find_all("a"):
  print(x)

# DOU

In [3]:
import os
from PyPDF2 import PdfReader
import requests as r
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [4]:
## ACESSAR
# https://www.in.gov.br/servicos/diario-oficial-da-uniao
## IR PARA BASES DE DADOS E PUBLICAÇÕES // DIÁRIO COMPLETO: VERSÃO CERTIFICADA
## BAIXAR OS PDFS E COLOCAR NA PASTA CORRESPONDENTE

#CAMINHO DO ARQUIVO
FILES_PATH = "/content/drive/MyDrive/DATA/DOU/SHARE"
#ARQUIVOS
FILES = [[x.name, 
          len(PdfReader(open(x.path, 'rb')).pages),
          PdfReader(open(x.path, 'rb'))
          ] for x in os.scandir(FILES_PATH) if ".pdf" in str(x)]
FILES


[['2023_01_18_ASSINADO_do2 (1).pdf',
  57,
 ['2023_01_18_ASSINADO_do2.pdf',
  57,
 ['2023_01_18_ASSINADO_do3.pdf',
  267,

[['2023_01_18_ASSINADO_do2 (1).pdf',
  57,
 ['2023_01_18_ASSINADO_do3.pdf',
  267,
 ['2023_01_18_ASSINADO_do2.pdf',
  57,

In [31]:
#NUMERO DE PAGINAS
files = []
for file in FILES:
  text = ""
  file_name = file[0]
  file_pgs = file[1]
  file_obj = file[2]
  for page in range(0,file_pgs):
    text = file_obj.pages[page].extract_text()
    text += " ".join(text.split("\n"))
    break
  files = [file_name, file_pgs, text]
files 

['2023_01_18_ASSINADO_do3.pdf',
 267,
 'REPÚBLICA FEDERATIVA DO BRASIL • IMPRENSA NACIONAL\nAno CLXI Nº 13 Brasília - DF, quarta-feira, 18 de janeiro de 2023ISSN 1677-7069\n3\nDocumento assinado digitalmente conforme MP nº 2.200-2 de 24/08/2001,\nque institui a Infraestrutura de Chaves Públicas Brasileira - ICP-Brasil.Este documento pode ser verificado no endereço eletrônico\nhttp://www.in.gov.br/autenticidade.html , pelo código 053020230118000011Presidência da República ................................................................ .......................................... 1\nMinistério da Agricultura, Pecuária e Abastecimento ............................................................ 2\nMinistério da Cidadania .......................................................................................................... 11\nMinistério da Ciência, Tecnologia e Inovações ................................................................\n..... 12\nMinistério das Comunicações ............

In [ ]:
for pg in FILES:
  for p in range(0, pg[2]):
    print(....)

In [ ]:

for pg in FILES:
  pgs = len(file.pages)
  text=''
  for p in range(0,pg[1]):
    # creating a page object
    pageObject = file.pages[p]
    # extracting text from page
    text += pageObject.extract_text()
    break
  lines = [[p, x, y] for  x, y in enumerate(text.split("\n"))]


lines = pd.DataFrame([x for x in lines])
lines.columns = ["pg_num", "enum", "line"]

display(lines)

,pg_num,enum,line
0,0,0,REPÚBLICA FEDERATIVA DO BRASIL • IMPRENSA NACI...
1,0,1,"Ano CLXI Nº 12 Brasília - DF, terça-feira, 17 ..."
2,0,2,3
3,0,3,Documento assinado digitalmente conforme MP nº...
4,0,4,que institui a Infraestrutura de Chaves Públic...
...,...,...,...
135,0,135,novecentos e dezenoce reais). O processo se en...
136,0,136,s.
137,0,137,RODRIGO DO CANTO CARDOSO
138,0,138,Pregoeiro


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("my_app") \
    .getOrCreate()

In [ ]:
data = spark.createDataFrame(lines)
data.show(10)

+------+----+--------------------+
|pg_num|enum|                line|
+------+----+--------------------+
|     0|   0|REPÚBLICA FEDERAT...|
|     0|   1|Ano CLXI Nº 12 Br...|
|     0|   2|                   3|
|     0|   3|Documento assinad...|
|     0|   4|que institui a In...|
|     0|   5|http://www.in.gov...|
|     0|   6|Ministério da Agr...|
|     0|   7|Ministério da Cid...|
|     0|   8|Ministério da Ciê...|
|     0|   9|           ....... 9|
+------+----+--------------------+
only showing top 10 rows



In [ ]:
import string
from string import punctuation
import pyspark.sql.functions as F

data = data.withColumn("texto_regex", F.regexp_replace("line", "[\$#,\"!%&'()*+-./:;<=>?@^_`´{|}~\\\\]", ""))
data = data.withColumn("texto_trim", F.trim(data.texto_regex))
data.show(10, False)

+------+----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+
|pg_num|enum|line                                                                                                                                                                                                          |texto_regex                                                                                                                    |texto_trim                                                                                                                     |
+------+----+---------------------------------

In [ ]:
amostra = data.select('texto_trim').sample(fraction = 0.5, seed = 101)
tudo = [texto['texto_trim'] for texto in amostra.collect()]
wordcloud = WordCloud(background_color = 'white',
                      width = 1000,
                      height = 600,
                      collocations = True, 
                      prefer_horizontal = 1).generate(str(tudo))
plt.figure(figsize=(20,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol = "texto_trim", outputCol = "tokens")
tokenizado = tokenizer.transform(data)

tokenizado.select("texto_trim", "tokens").show()

+--------------------+--------------------+
|          texto_trim|              tokens|
+--------------------+--------------------+
|REPÚBLICA FEDERAT...|[república, feder...|
|Ano CLXI Nº 12 Br...|[ano, clxi, nº, 1...|
|                   3|                 [3]|
|Documento assinad...|[documento, assin...|
|que institui a In...|[que, institui, a...|
|httpwwwingovbraut...|[httpwwwingovbrau...|
|Ministério da Agr...|[ministério, da, ...|
|Ministério da Cid...|[ministério, da, ...|
|Ministério da Ciê...|[ministério, da, ...|
|                   9|                 [9]|
|Ministério das Co...|[ministério, das,...|
|Ministério da Def...|[ministério, da, ...|
|Ministério do Des...|[ministério, do, ...|
|                  23|                [23]|
|Ministério da Eco...|[ministério, da, ...|
|Ministério da Edu...|[ministério, da, ...|
|Ministério da Inf...|[ministério, da, ...|
|                  77|                [77]|
|Ministério da Jus...|[ministério, da, ...|
|Ministério do Mei...|[ministéri